In [6]:
import pickle
import glob
import time

#import imageio as im
from scipy.misc import imread

import random as rd
from random import randint, uniform, sample, seed

import tensorflow as tf
import numpy as np

from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D 
from keras.models import Model, load_model

import keras.backend as K

import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['imread', 'randint', 'sample', 'seed', 'uniform']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


### Основные функции

In [27]:
def Recognizer_v1(input_shape = (58, 58, 3), classes = 12):
    """
    Keras CNN
    INPUT -> CONV -> RELU -> MAXPOOL -> CONV -> RELU -> FC -> RELU -> OUTPUT
    """
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(16, (5, 5), strides = (1, 1), name = 'conv0')(X_input)
    X = Activation('relu')(X)
    
    # Maxpool
    X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X)
    
    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(16, (3, 3), strides = (1, 1), name = 'conv1')(X)
    X = Activation('relu')(X)
    
    # FLATTEN X (means convert it to a vector) + FC + RELU
    X = Flatten()(X)
    X = Dense(256, name='fc0')(X)
    X = Activation('relu')(X)
    
    # FULLYCONNECTED
    X = Dense(classes, name='fc1')(X)
    X = Activation('softmax')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='Recognizer')

    return model


### Main program body

In [32]:
tmp = []

img = imread("Plates_Train\\red_3.png")
tmp.append(img)
img = imread("Plates_Train\\red_4.png")
tmp.append(img)
img = imread("Plates_Train\\red_5.png")
tmp.append(img)

img = imread("Plates_Train\\green_3.png")
tmp.append(img)
img = imread("Plates_Train\\green_4.png")
tmp.append(img)
img = imread("Plates_Train\\green_5.png")
tmp.append(img)

img = imread("Plates_Train\\cyan_3.png")
tmp.append(img)
img = imread("Plates_Train\\cyan_4.png")
tmp.append(img)
img = imread("Plates_Train\\cyan_5.png")
tmp.append(img)

img = imread("Plates_Train\\purple_3.png")
tmp.append(img)
img = imread("Plates_Train\\purple_4.png")
tmp.append(img)
img = imread("Plates_Train\\purple_5.png")
tmp.append(img)

X_train = np.stack(tmp, axis=0)
Y_train = np.eye(12)

In [ ]:
# CLASSES:
# ========
#
# Red 3 = 0
# Red 4 = 1
# Red 5 = 2
# 
# Green 3 = 3
# Green 4 = 4
# Green 5 = 5
# 
# Cyan 3 = 6
# Cyan 4 = 7
# Cyan 5 = 8
# 
# Purple 3 = 9
# Purple 4 = 10
# Purple 5 = 11

In [39]:
# Готовим модель
recon = Recognizer_v1(input_shape=(58, 58, 3), classes=12)
recon.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#recon.summary()

In [40]:
%%time
recon.fit(X_train, Y_train, epochs=50, verbose=1)

# Score trained model
scores = recon.evaluate(X_train, Y_train, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/50
12/12 [==============================] - 1s 52ms/step - loss: 2.4640 - acc: 0.0833
Epoch 2/50
12/12 [==============================] - 0s 7ms/step - loss: 1.9736 - acc: 0.6667
Epoch 3/50
12/12 [==============================] - 0s 8ms/step - loss: 1.3127 - acc: 0.7500
Epoch 4/50
12/12 [==============================] - 0s 7ms/step - loss: 0.7605 - acc: 1.0000
Epoch 5/50
12/12 [==============================] - 0s 9ms/step - loss: 0.4255 - acc: 0.9167
Epoch 6/50
12/12 [==============================] - 0s 9ms/step - loss: 0.2585 - acc: 1.0000
Epoch 7/50
12/12 [==============================] - 0s 10ms/step - loss: 0.1513 - acc: 1.0000
Epoch 8/50
12/12 [==============================] - 0s 8ms/step - loss: 0.0864 - acc: 1.0000
Epoch 9/50
12/12 [==============================] - 0s 6ms/step - loss: 0.0713 - acc: 1.0000
Epoch 10/50
12/12 [==============================] - 0s 11ms/step - loss: 0.0544 - acc: 1.0000
Epoch 11/50
12/12 [==============================] - 0s 7ms/step -

In [41]:
#
# Very Important!!!
#
recon.save("Recognizer_v1.h5")
